# Project milestone 4
Tadaa - Jonathan Haenni, Lea Schmidt, Danny Kohler
> This work presents the creative extension of the publication from J. Penney : "Chilling effects : Online surveillance and Wikipedia use".

---

## Abstract
The goal of this work is to observe if the public action of Greta Thunberg and her continual incitation for people to educate themselves on the subject of climate change has actually made a difference. This effect would be diametrically different from a chilling effect in that it pushes people to get educated on a subject, effectively arguably increasing their levels of freedom. This effect shall henceforth be called an "Empowering effect".

First and foremost, the selected interest period depends on the choice of events that will be the pivots of the interrupted time series analysis. (ITS) These events are as follows: the school strike of August 20, 2018 led by Greta Thunberg, the 2018 United Nations Climate Change Conference (COP24) on December 14, 2018 and the summit of the United Nations on September 23, 2019. The period of analysis is between January 2018 and February 2020. As seen in HW1, the Covid-19 pandemic influences the pageviews a lots and we will therefore not elongate this analysis onto 2020.

The data considered here comes from Wikipedia. We will consider 150 [ADAPT VALUE] Wikipedia articles divided into 3 groups. The first group is the treatment dataset, containing the Wikipedia articles related to climate change issues. The second group is a quasi-control group, where the topics considered are related to nature, without being related to climate change directly. The third group is another control group composed of popular articles simply reflecting the trends on Wikipedia. It is the same as the one used in the publication. The data sets considered give the number of pageviews per day for each article, within the time period considered.

As already hinted at, the analysis will be very similar to the one used in the publication - an ITS analysis with segmented linear regressions.

If the first group features significant changes in pageviews and the control groups don't across the selected period, we will be able to conclude that the "Empowering Effect" exists and we will be able to compare it to the chilling effect considered in the publication.

---

## 1. Data loading and filtering

> code description

In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

# Load the climate change articles data
art_cc1 = pd.read_csv('data/pageviews-10-pro.csv').copy()
art_cc2 = pd.read_csv('data/pageviews-20-pro.csv').copy()
art_cc3 = pd.read_csv('data/pageviews-30-pro.csv').copy()
art_cc4 = pd.read_csv('data/pageviews-40-pro.csv').copy()
art_cc5 = pd.read_csv('data/pageviews-50-pro.csv').copy()

# Load the control (popular) articles data
art_control1 = pd.read_csv('data/pageviews-10-control.csv').copy()
art_control2 = pd.read_csv('data/pageviews-20-control.csv').copy()
art_control3 = pd.read_csv('data/pageviews-30-control.csv').copy()
art_control4 = pd.read_csv('data/pageviews-40-control.csv').copy()
#art_control5 = pd.read_csv('data/pageviews-50-control.csv').copy()

# Merge all datasets together
art_cc=art_cc1.merge(art_cc2).merge(art_cc3).merge(art_cc4).merge(art_cc5)
art_control=art_control1.merge(art_control2).merge(art_control3).merge(art_control4)#.merge(art_control5)

# Display to illustrate the structure
art_control.head()

In [ ]:
# # Translate dates in datetime type
# art_cc.Date=pd.to_datetime(art_cc.Date)
# art_control.Date=pd.to_datetime(art_control.Date)

# # Set the date as the index
# art_cc=art_cc.set_index('Date')
# art_control=art_control.set_index('Date')

# # Agregate the data by months
# art_cc=art_cc.groupby(pd.Grouper(freq="M")).sum()
# art_control=art_control.groupby(pd.Grouper(freq="M")).sum()

# # Display to illustrate the structure
# display(art_control.head())

# # At this point, we should  sum over all articles.
# art_cc=art_cc.sum(axis=1)
# art_control=art_control.sum(axis=1)

# # Here the index is reset to get the number of months instead of dates, as in the publication.
# # The date column is then removed : it is not needed anymore
# # The pageviews column is renamed
# art_cc=art_cc.reset_index()
# art_cc=art_cc.drop('Date', axis=1)
# art_cc.columns=['pageviews']

# art_control=art_control.reset_index()
# art_control=art_control.drop('Date', axis=1)
# art_control.columns=['pageviews']

# # Display to illustrate the structure
# display(art_control.head())

>Here we have a function to aggregate the dataframe. It creates a dataframe with the index as the datetimes by month and with two columns as the number of the month and the pageviews count over all the article during the said month. 

In [ ]:
#Je propose cette fonction pour aggréger directement mais en pouvant choisir quel article drop 
#s'il y a des outliers. En plus les dates sont en index ce qui permet d'utiliser les features de Pandas

def aggregate(df, freq="M", drop = None) :
    """A function to aggregate the dataframe as a Serie sorted by month
    with the total number of pageviews per month. With the numbering of each
    month of the timeserie. drop is used to drop a list of articles in the 
    dataframe. freq is used to choice the frequency of the aggregation. Return the dataframe"""
    df=df.copy()
    df.index=df.Date
    df.index=pd.to_datetime(df.index)
    df=df.drop(columns="Date")
    #Recreate the dataframe without the named articles
    if not drop == None :
        df=df.drop(columns=drop)
    
    # At this point, we should  sum over all articles.
    df=df.sum(axis=1)
    # Agregate the data by months
    df=df.groupby(pd.Grouper(freq=freq)).sum()
    #Serie to dataframe
    df=df.to_frame()
    #Rename the column
    df.columns=['pageviews']
    #Add the column months to help to plot with the regression
    months=np.arange(1, len(df.index)+1)
    df["months"]=months
    return df

art_control_agg=aggregate(df=art_control)
art_cc_agg=aggregate(df=art_cc)

display(art_control_agg)

---

## 2. Computing the Segmented Regression parameters

>Here we define all the intervals between the mentioned event about the climate : the school strike of August 20, 2018 led by Greta Thunberg and the 2018 United Nations Climate Change Conference (COP24) on December 14, 2018.

In [ ]:
INTERVALS =[["2018-01","2018-07"],["2018-08","2018-07"],["2018-01","2018-07"],["2018-01","2018-07"]]

In [ ]:
import statsmodels.formula.api as smf
import datetime
from statsmodels.stats.outliers_influence import summary_table

def list_LR_periods(df, before_interval=["2012-01","2013-05"], after_interval=["2013-06","2014-08"], CI=0.05) :
    """"""
    
    #Linear regression before June 2013  
    mod_before = smf.ols(formula="pageviews ~ months", data=df.loc[before_interval[0]:before_interval[1]])
    res_before=mod_before.fit()
    #Compute the serie of predicted values from the basic linear equation with one variable : y = intercept + coef * x
    pages_predict_before=res_before.params[0]+res_before.params[1]*df.loc[before_interval[0]:before_interval[1]]["months"]
    #Extract the computed values (including upper and lower CI values)
    _, summary_values_before, summary_names_before = summary_table(res_before, alpha=CI)
    #Create a temporary dataframe with the result
    df_res_tmp = pd.DataFrame(summary_values_before, columns=summary_names_before)
    display(df_res_tmp)
    #WARNING : It's a bit tricky but the column has always 95% in its name even if the CI
    #introduce in alpha is different of 0.05. It has been tested and the values are different.
#     predict_ci_low_before = df_res_tmp["Predict ci\n95% low"].T
#     predict_ci_upp_before = df_res_tmp["Predict ci\n95% upp"].T
    predict_ci_low_before = df_res_tmp["Mean ci\n95% low"].T
    predict_ci_upp_before = df_res_tmp["Mean ci\n95% upp"].T

    #Linear regression after June 2013
    mod_after = smf.ols(formula="pageviews ~ months", data=df.loc[after_interval[0]:after_interval[1]])
    res_after=mod_after.fit()
    #Compute the serie of predicted values from the basic linear equation with one variable : y = intercept + coef * x
    pages_predict_after=res_after.params[0]+res_after.params[1]*df.loc[after_interval[0]:after_interval[1]]["months"]
    #Extract the computed values (including upper and lower CI values)
    _, summary_values_after, summary_names_after = summary_table(res_after, alpha=CI)
    #Create a temporary dataframe with the result
    df_res_tmp = pd.DataFrame(summary_values_after, columns=summary_names_after)
#     predict_ci_low_after = df_res_tmp["Predict ci\n95% low"].T
#     predict_ci_upp_after = df_res_tmp["Predict ci\n95% upp"].T
    predict_ci_low_after = df_res_tmp["Mean ci\n95% low"].T
    predict_ci_upp_after = df_res_tmp["Mean ci\n95% upp"].T
    
    #Create a dictionnary to pass the results
    dic = {
        "before" :{
            "result" : res_before,
            "predicts" : pages_predict_before,
            "CI" : {
                "lower" : predict_ci_low_before,
                "upper" : predict_ci_upp_before
            }
        }, 
        "after" : {
            "result" : res_after,
            "predicts" : pages_predict_after,
            "CI" : {
                "lower" : predict_ci_low_after,
                "upper" : predict_ci_upp_after
            }
        }
    }
    return dic

---

## 3. Displaying the results

> code description

In [ ]:
#CODE